In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import chardet
#df = pd.read_csv("sales_total_sku_channel.csv",low_memory=False)
%matplotlib inline

## 货品维度

#### 考虑该货品的品类，颜色，推出至今的年份（考虑到款式可能渐渐过时等），标牌价格，售卖价格，折扣率，地区（气候与消费能力），材料，系列等
#### 对销售量，尺码比例等进行预测


In [2]:
goods = pd.read_csv("goods.csv")

In [3]:
goods.columns

Index(['sale_date', 'category_sub', 'year', 'color_name', 'sale_price',
       'tag_price', 'season', 'series', 'province', 'material', 'size_name',
       'Quantity', 'discount', 'release_years', 'month', '季节', 'sku_id',
       '周数-年'],
      dtype='object')

 周数-年是指销售日期是这一年的第几周，release_years是销售的年份减去发行的年份，seaon是原数据提供的，应该是指品类对应的季节，“季节”是销售日期的季节

'release_years' 是销售年份减去发行年份
discount是 销售价格除以标牌价格

## 渠道维度

#### 除了考虑前面的属性，还要考虑门店的面积，导购员数量，地址（可以归总地址是否热门）
#### 对销售量，尺码比例等进行预测

In [4]:
df_channel = pd.read_csv("goods_channel.csv")

In [5]:
df_channel.columns

Index(['sale_date', 'category_sub', 'year', 'color_name', 'sale_price',
       'tag_price', 'season', 'series', 'province', 'area',
       'shopping_guide_quantity', 'address', 'channel_type', 'channel_name',
       'material', 'size_name', 'Quantity', 'release_years', '季节', 'sku_id',
       '周数-年'],
      dtype='object')

In [7]:
group = df_channel.groupby(by = "channel_name")

In [11]:
for key,item in group:
    group.get_group(key).to_csv("店铺/"+key.replace("/","_")+".csv",index = False)
    

按照店铺分别输出了csv文件

In [12]:
group_year = df_channel.groupby(by = "year")

In [15]:
for key,item in group_year:
    group_year.get_group(key).to_csv("年份/"+str(key)+".csv",index = False)

按照年份分别输出了csv文件

In [35]:
df_channel['channel_name'].head(5)

0    重庆王府井百货
1     深圳太阳百货
2      南京水游城
3     深圳太阳百货
4    重庆王府井百货
Name: channel_name, dtype: object

可以根据同一年份同一地区的不同地址销售额度来看地址是否有影响， 这边的销售额可以定义为 总销售额/面积 等，进行规整

### 订单的归总

In [17]:
purchase = pd.read_csv("New_data/purchase_20190410104706.csv")

In [25]:
df_sku = goods[['sku_id','season','series','category_sub','color_name','tag_price','material', 'size_name']].groupby(by = 'sku_id').head(1)

In [26]:
df_sku.head(5)

,sku_id,season,series,category_sub,color_name,tag_price,material,size_name
0,121802023B701#B701#00#L,春季,Modern City,夹克,黑色,699.0,皇家细格尼丝纺,L
97,112804029B365#B365#00#M,夏季,Other,T恤,薄荷绿,199.0,NaN,M
167,121904087B902#B902#00#M,春季,Modern City,T恤,麻灰,229.0,32'S 单纱汗布,M
248,1218020032B01#2B01#00#M,春季,Denim,夹克,蓝牛,799.0,美术牛仔,M
530,1219070042B01#2B01#32#29,春季,Denim,牛仔长裤,蓝牛,599.0,弹力黛林牛仔,29


In [27]:
new_df = pd.merge(purchase,df_sku,on="sku_id")

In [28]:
new_df.head(5)

,purchase_no,purchase_date,supplier_id,sku_id,purchase_price,Quantity,season,series,category_sub,color_name,tag_price,material,size_name
0,POB0000002,2013/06/03,GYS074,133894008609#609#00#L,61.0,121,秋季,Modern Vintage,配件,藏黑蓝,349.0,牛皮,L
1,POB0000002,2013/06/03,GYS074,133894009402#402#00#M,79.0,93,秋季,Modern Vintage,配件,咖啡棕,349.0,牛皮,M
2,POB0000002,2013/06/03,GYS074,133994002104#104#00#M,70.0,114,秋季,Modern Vintage,配件,果酱红,299.0,牛皮,M
3,POB0000002,2013/06/03,GYS074,133994002104#104#00#S,70.0,87,秋季,Modern Vintage,配件,果酱红,299.0,牛皮,S
4,POB0000002,2013/06/03,GYS074,134894001402#402#00#L,70.0,116,冬季,Modern Vintage,配件,咖啡棕,349.0,牛皮,L


自此可以汇总订单量以及利用货品的属性预测订单量，或者统计订单量的尺寸比例等.也可以根据季节等。 这里的season是货品对应季节，再对应采购日期的季节或许会有帮助

In [29]:
def season(i):
    if i in [3,4,5]:
        return "春"
    if i in [6,7,8]:
        return "夏"
    if i in [9,10,11]:
        return "秋"
    if i in [12,1,2]:
        return "冬"
    return np.nan

In [32]:
new_df["purchase_month"] = pd.to_datetime(new_df["purchase_date"]).apply(lambda x : x.strftime("%m")).astype(int)
new_df["采购季节"]=new_df["purchase_month"].apply(season)

In [36]:
new_df["purchase_year"] = pd.to_datetime(new_df["purchase_date"]).apply(lambda x : x.strftime("%Y")).astype(int)

In [37]:
new_df.head(5)

,purchase_no,purchase_date,supplier_id,sku_id,purchase_price,Quantity,season,series,category_sub,color_name,tag_price,material,size_name,purchase_month,采购季节,purchase_year
0,POB0000002,2013/06/03,GYS074,133894008609#609#00#L,61.0,121,秋季,Modern Vintage,配件,藏黑蓝,349.0,牛皮,L,6,夏,2013
1,POB0000002,2013/06/03,GYS074,133894009402#402#00#M,79.0,93,秋季,Modern Vintage,配件,咖啡棕,349.0,牛皮,M,6,夏,2013
2,POB0000002,2013/06/03,GYS074,133994002104#104#00#M,70.0,114,秋季,Modern Vintage,配件,果酱红,299.0,牛皮,M,6,夏,2013
3,POB0000002,2013/06/03,GYS074,133994002104#104#00#S,70.0,87,秋季,Modern Vintage,配件,果酱红,299.0,牛皮,S,6,夏,2013
4,POB0000002,2013/06/03,GYS074,134894001402#402#00#L,70.0,116,冬季,Modern Vintage,配件,咖啡棕,349.0,牛皮,L,6,夏,2013


In [38]:
new_df.to_csv("purchase_info.csv",index = False)

选出2013年的来做一下

In [41]:
new_df["purchase_year"].value_counts()

2017    13212
2014     9480
2016     6140
2015     5686
2018     4643
2013     4571
Name: purchase_year, dtype: int64

In [43]:
purchase_2013 = new_df[new_df["purchase_year"]==2013]

这里的折扣率就用2013年全年的折扣率（销售额/（销售数量×吊牌价））吧，虽然这样不严谨。

In [45]:
df_2013 = pd.read_csv("年份/2013.csv")

In [48]:
df_2013.columns

Index(['sale_date', 'category_sub', 'year', 'color_name', 'sale_price',
       'tag_price', 'season', 'series', 'province', 'area',
       'shopping_guide_quantity', 'address', 'channel_type', 'channel_name',
       'material', 'size_name', 'Quantity', 'release_years', '季节', 'sku_id',
       '周数-年'],
      dtype='object')

In [49]:
df_2013["sales"] = df_2013["sale_price"]*df_2013["Quantity"]

In [50]:
df_2013["sales_without_discount"]=df_2013["tag_price"]*df_2013["Quantity"]#如果没有折扣，会是多少钱，用来统计折扣率

### 这里的销售额以及原价情况下的销售额也是一整年的

In [54]:
group_2013 = df_2013[['sku_id','sales','sales_without_discount','Quantity']].groupby(by = "sku_id")

In [56]:
sales_2013=group_2013.sum()

In [57]:
sales_2013.head(3)

,sales,sales_without_discount,Quantity
sku_id,,,
0718010020314#0314#30#28,869.0,1198.0,2.0
0718010020314#0314#30#32,307.0,349.0,1.0
0718020010322#0322#00#XL,1438.0,1798.0,2.0


In [59]:
sales_2013.reset_index(inplace = True)

In [60]:
sales_2013.head(3)

,sku_id,sales,sales_without_discount,Quantity
0,0718010020314#0314#30#28,869.0,1198.0,2.0
1,0718010020314#0314#30#32,307.0,349.0,1.0
2,0718020010322#0322#00#XL,1438.0,1798.0,2.0


In [61]:
sales_2013["折扣率"]=sales_2013["sales"]/sales_2013["sales_without_discount"]

In [62]:
sales_2013.head(3)

,sku_id,sales,sales_without_discount,Quantity,折扣率
0,0718010020314#0314#30#28,869.0,1198.0,2.0,0.725376
1,0718010020314#0314#30#32,307.0,349.0,1.0,0.879656
2,0718020010322#0322#00#XL,1438.0,1798.0,2.0,0.799778


In [63]:
purchase_2013_ = pd.merge(purchase_2013,sales_2013,on="sku_id")

In [64]:
purchase_2013_.head(3) 

,purchase_no,purchase_date,supplier_id,sku_id,purchase_price,Quantity_x,season,series,category_sub,color_name,tag_price,material,size_name,purchase_month,采购季节,purchase_year,sales,sales_without_discount,Quantity_y,折扣率
0,POB0000002,2013/06/03,GYS074,134894001402#402#00#L,70.00,116,冬季,Modern Vintage,配件,咖啡棕,349.0,牛皮,L,6,夏,2013,19238.5,26810.0,40.0,0.717587
1,POB0000004,2013/06/03,GYS079,133798003611#611#00#38,184.15,93,秋季,Modern Vintage,配件,靛蓝,799.0,巴布牛仔,38,6,夏,2013,17340.5,23370.0,30.0,0.741998
2,POB0000004,2013/06/03,GYS079,133798003611#611#00#42,184.15,149,秋季,Modern Vintage,配件,靛蓝,799.0,巴布牛仔,42,6,夏,2013,18108.0,35658.0,42.0,0.507824


In [66]:
purchase_2013.to_csv("purchase_2013.csv", index =False)

然后可以用groupby来计算每个category_sub下面的sku的数量，分别的价格，折扣率，全年的销售额等等